In [1]:
import ee


In [4]:

##ee.Authenticate()
ee.Initialize()
import geemap
Map = geemap.Map(center=[36,-120], zoom =8)


In [5]:
#AOI
aoi = ee.Geometry.BBox(-121.351882, 36.480156 , -120.861284,36.874108)
#Start geemap 
Map = geemap.Map(center=[36,-120], zoom =8)


bps = ee.ImageCollection("LANDFIRE/Vegetation/BPS/v1_4_0").reduce(ee.Reducer.max())
#rapcovers = ee.ImageCollection("LANDFIRE/Vegetation/EVH/v1_4_0").reduce(ee.Reducer.max())
evh = ee.ImageCollection("LANDFIRE/Vegetation/EVH/v1_4_0").reduce(ee.Reducer.max())
rapcoverha = ee.Image("projects/rap-data-365417/assets/vegetation-cover-v3/2014").select('AFG')
rapcoverhp = ee.Image("projects/rap-data-365417/assets/vegetation-cover-v3/2014").select('PFG') 
sclass = ee.ImageCollection("LANDFIRE/Fire/SClass/v1_4_0").reduce(ee.Reducer.max())

rapcoverh = rapcoverha.add(rapcoverhp)

col = ee.ImageCollection([])
for year in range(2019,2024):
    # Define the GeoTIFF file path in the GCS bucket
    geotiff_path = f'gs://gigafire_rvs/rpms/rpms_{year}.tif' 
    
    # Load the GeoTIFF as an image
    image = ee.Image.loadGeoTIFF(geotiff_path)
    
    # Add the image to the image collection
    col = col.merge(ee.ImageCollection([image]))

# Print the image collection
print("Image Collection:", col.getInfo())


percentiles = [5, 25, 50, 75, 95]


rpms_percentiles = col.reduce(ee.Reducer.percentile(percentiles))

col = ee.ImageCollection("LANDSAT/LC08/C01/T1_32DAY_NDVI")

def annual_composite(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    annual_ndvi = col.filterDate(start_date, end_date).median().set('year', year)
    return annual_ndvi
    
    # Create annual composites for each year in the range
start_year = 2013
end_year = 2021
annual_ndvi_collection = ee.ImageCollection.fromImages(
    [annual_composite(year) for year in range(start_year, end_year + 1)])

# Compute percentiles
ndvi_percentiles = annual_ndvi_collection.reduce(ee.Reducer.percentile(percentiles))

#Gridmet precip
gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select(['pr'])

#Gridmet annual sums
def annual_composite(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    annual_precip = gridmet.filterDate(start_date, end_date).sum().set('year', year)
    return annual_precip
    
    # Create annual composites for each year in the range
start_year = 1979
end_year = 2023
annual_precip_collection = ee.ImageCollection.fromImages(
    [annual_composite(year) for year in range(start_year, end_year + 1)])

precip_percentiles = annual_precip_collection.reduce(ee.Reducer.percentile(percentiles))


landfire_stack = ee.Image(
        [bps, rapcoverh, evh,ndvi_percentiles, precip_percentiles, sclass,rpms_percentiles]).toFloat()






In [7]:
from google.cloud import storage

bucket_id = "gigafire_rvs"

client = storage.Client()
bucket = client.get_bucket(bucket_id)
# list all objects in the directory
blobs = bucket.list_blobs()
for blob in blobs:
    blob.delete()

#Export tiles to bucket (shard size 256)
def export_stack_to_cloud_storage(stack):
    # ee_region_bbox = ee_region.bounds()
    # print(ee_region_bbox.coordinates())
    task = ee.batch.Export.image.toCloudStorage(
        image=stack,
        fileNamePrefix="lf",
        bucket=bucket_id,
        scale=30,
        crs='EPSG:4326',
        shardSize=256,
        region=aoi,
        fileDimensions=256,
        skipEmptyTiles=True,
        maxPixels=1e13,
        fileFormat="GeoTIFF",
        formatOptions={
            'cloudOptimized': True
        }
    )
    task.start()

export_stack_to_cloud_storage(landfire_stack)


In [8]:
###Download from bucket to tmp folder

from google.cloud import storage
import os
import tempfile

bucket_id = "gigafire_rvs"
#wd = "C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/temp"
# Create a temporary directory to download the files
temp_dir = tempfile.mkdtemp()
print(temp_dir)

def list_files_in_bucket(bucket_name):
    """List all files in a Google Cloud Storage bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    files = []
    blobs = bucket.list_blobs()
    for blob in blobs:
        files.append(blob.name)

    return files

def download_files_from_bucket(bucket_name, local_temp_dir):
    """Download files from a Google Cloud Storage bucket to a local directory."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    local_files = []
    blobs = bucket.list_blobs(prefix="lf")
    for blob in blobs:
        local_file_path = os.path.join(local_temp_dir, blob.name)
        blob.download_to_filename(local_file_path)
        local_files.append(local_file_path)

    return local_files




# Download files to temporary directory
local_files = download_files_from_bucket(bucket_id, temp_dir)
print("Downloaded files:", local_files)





C:\Users\ingli\AppData\Local\Temp\tmpi0jtikef
Downloaded files: ['C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000000000.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000000256.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000000512.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000000768.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000001024.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000001280.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000001536.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000000-0000001792.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000256-0000000000.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000256-0000000256.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpi0jtikef\\lf0000000256-0000000512.tif', 'C:\\Users\

In [9]:
##Geotiff to dataframe

import rasterio
import pandas as pd




def multiband_geotiff_to_dataframe(geotiff_path):
    """Converts a multiband GeoTIFF file into a pandas DataFrame with x and y coordinates."""
    # Open the GeoTIFF file
    with rasterio.open(geotiff_path) as src:
        # Read all bands
        num_bands = src.count
        band_data = [src.read(i) for i in range(1, num_bands + 1)]
        
        # Get metadata
        transform = src.transform
        height, width = band_data[0].shape

        # Generate x and y coordinates
        x_coords = []
        y_coords = []
        for i in range(height):
            for j in range(width):
                x, y = src.xy(i, j)
                x_coords.append(x)
                y_coords.append(y)

        # Flatten band data and coordinates into DataFrame
        df_data = {'x': x_coords, 'y': y_coords}
        for band_idx in range(num_bands):
            band_name = src.descriptions[band_idx] if src.descriptions else f'band_{band_idx + 1}'
            df_data[band_name] = band_data[band_idx].flatten()

        df = pd.DataFrame(df_data)

    return df


dfs = [multiband_geotiff_to_dataframe(file_path) for file_path in local_files]

for df in dfs:
    # Identify NDVI columns
    ndvi_columns = [col for col in df.columns if col.startswith('NDVI_')]
    
    # Apply the power operation to NDVI columns
    if ndvi_columns:  # Check if there are any NDVI columns
        df[ndvi_columns] = df[ndvi_columns].apply(lambda x: x * 10000)

print(dfs[1].head())


# Clean up temporary directory
for file_path in local_files:
    os.remove(file_path)
os.rmdir(temp_dir)

# Concatenate the list of DataFrames into a single DataFrame
#combined_df = pd.concat(dfs, ignore_index=True)



# Raise values in selected columns to the power of 4

#Take subset 

# Display the DataFrame
#print(combined_df.head())


            x          y  BPS_max   AFG  EVH_max      NDVI_p5     NDVI_p25  \
0 -121.282939  36.874001    950.0  67.0    104.0  1816.177612  2155.356445   
1 -121.282669  36.874001    950.0  56.0    104.0  2516.092773  2751.035156   
2 -121.282400  36.874001    950.0  44.0    107.0  2516.092773  2751.035156   
3 -121.282130  36.874001    950.0  44.0    104.0  3471.396484  3791.721191   
4 -121.281861  36.874001    950.0  58.0    104.0  2678.726807  3007.115723   

      NDVI_p50     NDVI_p75     NDVI_p95       pr_p5      pr_p25      pr_p50  \
0  2365.049316  2841.971436  3011.120605  209.836014  323.512482  407.297546   
1  3005.544434  3017.038818  3365.416504  209.836014  323.512482  407.297546   
2  3005.544434  3017.038818  3365.416504  209.836014  323.512482  407.297546   
3  3886.108398  4047.327637  4687.019531  209.836014  323.512482  407.297546   
4  3151.767334  3272.114990  4484.061035  209.836014  323.512482  407.297546   

       pr_p75      pr_p95  SClass_max  
0  495.700

In [10]:
##Join df to kattribute table and reorder and clean columns
key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/LF_140BPS_01122015.csv")
height_key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/LF_140EVH_05092014.csv")

def join_new_attributes(df):  
    join = df.join(key[['VALUE','BPS_CODE', 'BPS_NAME','BPS_MODEL']].set_index('VALUE'), on='BPS_max')
    join = join.join(height_key[['VALUE','height']].set_index('VALUE'), on='EVH_max', how="outer")
    join = join.assign(PLOT_ID=range(len(join)))
    cols = list(join.columns)
    print(join.head())
    Plots_Active = join[['PLOT_ID', 'BPS_CODE', 'BPS_NAME', 'BPS_MODEL', 
                     'AFG', 'height',
                     'y', 'x', *cols[10:15], *cols[5:10], 'SClass_max']]
    return Plots_Active


dfs = [join_new_attributes(df) for df in dfs]
print(len(dfs))

KeyError: "['height'] not in index"

In [42]:
import sqlite3
import pandas as pd
import os
import shutil
import time

# Copy template to home folder for reading and writing 
source_file = 'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/test/rvs_demo_fromRobbAddFieldTypeMissing.db'
counter=0

for i in dfs:
    counter=counter+1
    destination_file = f'C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_{counter}.db'
    shutil.copy(source_file, destination_file)
    print("Template DB copied successfully.")
# Path to the SQLite database file
    database_path = f'C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_{counter}.db'
    print(database_path)

# Create a connection to the database
    conn = sqlite3.connect(database_path)

# Define the table name
    table_name = "Plots_Active"

# Get the column names of the table
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    cols= [column[1] for column in cursor.fetchall()]
    print(cols)
# Read data from pandas DataFrame
# Assuming df is your pandas DataFrame with the same column names


    i.columns=cols
    # Replace values in the table with pandas DataFrame values
    
    i.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.commit()

    table_name="Shrubs_Active"
    cursor.execute(f"DELETE FROM {table_name}")

    table_name="Disturbance_Plots_NoFire"
    cursor.execute(f"DROP TABLE {table_name}")

    table_name="Dist_year7fire"
    cursor.execute(f"DROP TABLE {table_name}")

# Commit changes and close connection
    conn.commit()
    conn.close()

#Copy file from home to RVS git folder for RVS use
    source_file = f'C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_{counter}.db'
    destination_file = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/inputs/240425_landfire_plots_cover_{counter}.db'
    shutil.copy(source_file, destination_file)
    print("Final DB copied successfully to inputs")


Template DB copied successfully.
C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_1.db
['PLOT_ID', 'BPS_CODE', 'BPS_NAME', 'BPS_MODEL', 'herb_cover', 'herb_height', 'latitude', 'longitude', 'PPT_1', 'PPT_2', 'PPT_3', 'PPT_4', 'PPT_5', 'NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4', 'NDVI_5', 'sclass']
Final DB copied successfully to inputs
Template DB copied successfully.
C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_2.db
['PLOT_ID', 'BPS_CODE', 'BPS_NAME', 'BPS_MODEL', 'herb_cover', 'herb_height', 'latitude', 'longitude', 'PPT_1', 'PPT_2', 'PPT_3', 'PPT_4', 'PPT_5', 'NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4', 'NDVI_5', 'sclass']


DatabaseError: Execution failed on sql 'DROP TABLE "Plots_Active"': database is locked

In [56]:
##ADD fire at YEAR 5 


database_path = f'C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_1.db'


# Create a connection to the database
conn = sqlite3.connect(database_path)

# Define the table name
    
n = dfs[0].shape[0]


# Create a cursor object using the cursor() method
cursor = conn.cursor()
cursor.execute("DELETE FROM Disturbance_Plots")
# Insert rows with PLOT_IDs from 1 to N
for plot_id in range(1, n + 1):
    insert_data_query = f'''
    INSERT OR REPLACE INTO Disturbance_Plots (PLOT_ID, DIST_TYPE, DIST_SUBTYPE, P1_VAL, FREQ, START_YEAR, STOP_YEAR)
    VALUES ({plot_id}, 'FIRE', 'WILDFIRE', 100.0, 0, 5, 5)
    '''
    cursor.execute(insert_data_query)
    cursor.connection.commit()

conn.commit()
# Close the connection
conn.close()


destination_file = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/inputs/240425_landfire_plots_cover_1_fire5.db'
shutil.copy(database_path, destination_file)
print("Final fire  DB copied successfully to inputs")


Final fire  DB copied successfully to inputs


In [33]:
rows = [200]

# DataFrame to store execution times
execution_times_df = pd.DataFrame(columns=['Rows', 'Execution Time (seconds)'])

import time
import pandas as pd

# Number of iterations

# DataFrame to store execution times
execution_times_df = pd.DataFrame(columns=['Rows', 'Execution Time (seconds)'])

for i in rows:
    
    source_file = 'C:/Users/ingli/Desktop/home/240425_landfire_plots_cover.db'
    destination_file = f'C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_{i}.db'
    shutil.copy(source_file, destination_file)
    print("File copied successfully.")

    conn = sqlite3.connect(destination_file)
    cursor = conn.cursor()
    

    cursor.execute(f'''
    DELETE FROM Plots_Active
    WHERE PLOT_ID NOT IN (
        SELECT PLOT_ID FROM Plots_Active
        ORDER BY PLOT_ID ASC
        LIMIT {i}
    )
    ''')
        
    conn.commit()
    conn.execute('VACUUM')
    # Close the database connection
    conn.close()


    source_file = f'C:/Users/ingli/Desktop/home/240425_landfire_plots_cover_{i}.db'
    destination_file = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/240425_landfire_plots_cover_{i}.db'
    shutil.copy(source_file, destination_file)
    print("File copied successfully.")

    source_file = f'/data/240425_landfire_plots_cover_{i}.db'



File copied successfully.
File copied successfully.


In [ ]:
print(execution_times_df)
#execution_times_df.set_index('Rows', inplace=True)
#execution_times_df.reset_index(inplace=True)
execution_times_df['Rows'] = pd.to_numeric(execution_times_df['Rows'])
execution_times_df.plot(x='Rows', y='Execution Time (seconds)', kind='line', title='RVS execution time vs. db size', xlabel='No. of rows', ylabel='Time (s)')

In [43]:
!docker run -v "C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test":"/data" rvs-image-2024 rvs /data/inputs/240425_landfire_plots_cover_1_fire5.db /data/outs/out_fire.db 10 true

^C


rvs
/data/inputs/240425_landfire_plots_cover_1.db
/data/outs/out.db
10
true
3.37.2
2022-01-06 13:25:41 872ba256cbf61d9290b571c0e6d82a20c224ca3ad82971edc46b29818d5dalt1
Loading records...
Done.

YEAR 0


YEAR 1


YEAR 2


YEAR 3


YEAR 4


YEAR 5


YEAR 6


YEAR 7


YEAR 8


YEAR 9


Ran to completion.
